# Day 9
Find the description of the problem [here](https://adventofcode.com/2025/day/9)!

## Part 1

Puzzle input:

In [205]:
with open("input_files/day_09.txt") as input_file:
    input = input_file.read()

Test input:

In [206]:
# # Comment this cell to use the puzzle input instead of the test input
# input = """7,1
# 11,1
# 11,7
# 9,7
# 9,5
# 2,5
# 2,3
# 7,3"""

Parse the input:

In [207]:
red_tiles = [tuple(map(int, line.split(","))) for line in input.splitlines()]

This first part is so easy...

_Edit from the future: I don't understand why this works, because the calculation of the area is wrong. It should be_ `(abs(x1 - x2) + 1) * (abs(y1 - y2) + 1)` _instead of_ `abs((x1 - x2 + 1) * (y1 - y2 + 1))`. _This was the reason for part 2 to fail, but somehow part 1 had a correct result. I wish it had failed on part 1, then I would have noticed it much earlier._

In [208]:
area = 0
for tile1 in red_tiles:
    for tile2 in red_tiles:
        if tile1 == tile2:
            continue
        x1, y1 = tile1
        x2, y2 = tile2
        new_area = abs((x1 - x2 + 1) * (y1 - y2 + 1))  # This line is wrong, but somehow it works
        if new_area > area:
            area = new_area
print(f"The largest area of a rectangle is {area}.")

The largest area of a rectangle is 4740155680.


## Part 2

...that it can only mean one thing: hellish second part!

I haven't been able to find any way to tackle this one (_so far_, I hope). I don't know how to detect the green tiles without brute force (which is not possible in the puzzle input), so I'm unable to detect whether a given rectangle is valid or not.

I've experimented a bit by checking whether a point corresponds to a right hand turn or a left hand turn, but I don't know where to take this or if this is helpful at all. 🙁

In [209]:
turns = []
for p1, p2, p3 in zip(red_tiles[-1:] + red_tiles[:-1], red_tiles, red_tiles[1:] + red_tiles[:1]):
    x1, y1 = p2[0] - p1[0], p2[1] - p1[1]
    x2, y2 = p3[0] - p2[0], p3[1] - p2[1]

    cross = x1 * y2 - y1 * x2

    if cross > 0:
        turns.append((p2, "right"))
    else:
        turns.append((p2, "left"))

Trying a different approach:

_Edit from the future: This box of code was preventing me from getting the correct value. Stupid area calculation, that was suposed to be the easy part!_

In [ ]:
# Get list of squares sorted by area
squares = []
for tile1 in red_tiles:
    for tile2 in red_tiles:
        if tile1 == tile2:
            continue
        x1, y1 = tile1
        x2, y2 = tile2
        area = (abs(x1 - x2) + 1) * (abs(y1 - y2) + 1)  # Fixed area calculation
        squares.append((area, (x1, y1), (x2, y2)))
squares.sort(reverse=True)

# Get list of all lines formed by connecting red tiles
lines = [(p1, p2) for p1, p2 in zip(red_tiles, red_tiles[1:] + red_tiles[:1])]

The following function calculates whether a point is inside the polygon defined by the path or not:

In [211]:
def is_inside(point, lines):
    x, y = point
    intersections = 0
    for (x1, y1), (x2, y2) in lines:
        if x == x1 == x2 and (y1 <= y <= y2 or y1 >= y >= y2):
            return True
        if y == y1 == y2 and (x1 <= x <= x2 or x1 >= x >= x2):
            return True
        
        if y1 == y2:
            continue

        if y1 < y < y2 or y1 > y > y2:
            if x <= x1:
                intersections += 1

    return intersections % 2 == 1

And this one calculates whether two lines intersect (but not on the edges):

In [212]:
def check_intersection(line1, line2):
    (x1, y1), (x2, y2) = line1
    (x3, y3), (x4, y4) = line2

    line1_vertical = x1 == x2
    line2_vertical = x3 == x4

    if line1_vertical and not line2_vertical:
        return (y1 > y3 > y2 or y1 < y3 < y2) and (x3 < x1 < x4 or x3 > x1 > x4)
    elif not line1_vertical and line2_vertical:
        return (x1 < x3 < x2 or x1 > x3 > x2) and (y3 < y1 < y4 or y3 > y1 > y4)
    return False

With these two functions, my code does work for the test input, but unfortunally I get the wrong answer for the real input.

the `is_inside` function does work correctly (in principle), but that's not enough as a rectangle could have all four corners within the polygon but an edge could cross a polygon line. That's the reason for the `check_intersection` function, which is probably the cause of the issue.

In [213]:
for i, square in enumerate(squares):
    p1 = square[1]
    p3 = square[2]
    p2 = (p3[0], p1[1])
    p4 = (p1[0], p3[1])

    line1 = (p1, p2)
    line2 = (p2, p3)
    line3 = (p3, p4)
    line4 = (p4, p1)

    this_square = True
    if not is_inside(p1, lines) or not is_inside(p2, lines) or not is_inside(p3, lines) or not is_inside(p4, lines):
        continue
    
    for line in lines:
        if check_intersection(line, line1) or check_intersection(line, line2) or check_intersection(line, line3) or check_intersection(line, line4):
            this_square = False
            break
 
    if this_square:
        print(f"The largest area of a rectangle using only green and red tiles is {square[0]}. (not correct)")
        break

The largest area of a rectangle using only green and red tiles is 2145225780. (not correct)


Another attempt, compacting the coordinates and then brute-forcing it. This works again for the test input, but not for the actual input. ~~I'm beginning to suspect that the issue is with the `is_inside` function...~~ I've been working on it some more, and I think I know the problem, but I don't know how to fix it:

The `is_inside` function should yield `True` if the ray cast to the right goes through an odd number of sides, and `False` if it goes through an even number of sides. The issue is that the edge cases └┐ and └┘ provide the same result (even number of intersections) although the first one should be odd. Anyway, I haven't found a way to fix this issue (yet?), as I would need to take into account multiple lines at once.

In [214]:
red_tiles_compacted = [(i, x, y) for i, (x, y) in enumerate(red_tiles)]
red_tiles_compacted = sorted(red_tiles_compacted, key=lambda x: x[1])
lookup_x = sorted(list(set([x for _, x, _ in red_tiles_compacted])))
red_tiles_compacted = sorted(red_tiles_compacted, key=lambda x: x[2])
lookup_y = sorted(list(set([y for _, _, y in red_tiles_compacted])))

squares_compacted = [(area, (lookup_x.index(tile1[0]), lookup_y.index(tile1[1])), (lookup_x.index(tile2[0]), lookup_y.index(tile2[1]))) for area, tile1, tile2 in squares]

lines_compacted = [((lookup_x.index(x1), lookup_y.index(y1)), (lookup_x.index(x2), lookup_y.index(y2))) for (x1, y1), (x2, y2) in lines]

for area, p1, p2 in squares_compacted:
    x_vals = [p1[0], p2[0]]
    y_vals = [p1[1], p2[1]]
    
    this_square = True
    for x in range(min(x_vals), max(x_vals) + 1):
        for y in range(min(y_vals), max(y_vals) + 1):
            if not is_inside((x, y), lines_compacted):
                this_square = False
                break
        if not this_square:
            break
    
    if this_square:
        print(f"The largest area of a rectangle using only green and red tiles is {area}. (not correct)")
        break

The largest area of a rectangle using only green and red tiles is 181356840. (not correct)


Final attempt, flood fill. ~~I expected this to work, but it doesn't either.~~


In [215]:
def flood_fill(inside, start_x, start_y):
    stack = [(start_x, start_y)]
    while stack:
        x, y = stack.pop()
        if (x, y) in inside:
            continue
        inside.add((x, y))
        stack.append((x + 1, y))
        stack.append((x - 1, y))
        stack.append((x, y + 1))
        stack.append((x, y - 1))
        if len(stack) > 1000000:
            raise MemoryError("Starting point is outside the polygon.")  # I know, this is bad. But I wanted to quick way to check

In [216]:
def line_to_points(line):
    (x1, y1), (x2, y2) = line
    if x1 == x2:
        return [(x1, y) for y in range(min(y1, y2), max(y1, y2) + 1)]
    elif y1 == y2:
        return [(x, y1) for x in range(min(x1, x2), max(x1, x2) + 1)]

In [217]:
red_tiles_compacted = [(i, x, y) for i, (x, y) in enumerate(red_tiles)]
red_tiles_compacted = sorted(red_tiles_compacted, key=lambda x: x[1])
lookup_x = sorted(list(set([x for _, x, _ in red_tiles_compacted])))
red_tiles_compacted = sorted(red_tiles_compacted, key=lambda x: x[2])
lookup_y = sorted(list(set([y for _, _, y in red_tiles_compacted])))

squares_compacted = [(area, (lookup_x.index(tile1[0]), lookup_y.index(tile1[1])), (lookup_x.index(tile2[0]), lookup_y.index(tile2[1]))) for area, tile1, tile2 in squares]
lines_compacted = [((lookup_x.index(x1), lookup_y.index(y1)), (lookup_x.index(x2), lookup_y.index(y2))) for (x1, y1), (x2, y2) in lines]

polygon = []
for line in lines_compacted:
    polygon += line_to_points(line)
polygon = set(polygon)

max_value = max(polygon, key=lambda x: x[0])[0]

flood_fill(polygon, int(max_value / 2), int(max_value / 4))

for area, p1, p2 in squares_compacted:
    x_vals = [p1[0], p2[0]]
    y_vals = [p1[1], p2[1]]
    
    this_square = True
    for x in range(min(x_vals), max(x_vals) + 1):
        for y in range(min(y_vals), max(y_vals) + 1):
            if (x, y) not in polygon:
                this_square = False
                break
        if not this_square:
            break
    
    if this_square:
        print(f"The largest area of a rectangle using only green and red tiles is {area}.")
        break

The largest area of a rectangle using only green and red tiles is 1543501936.


~~This Jupyter Notebook is proof of my descent to madness with this puzzle. You win.~~

**I win!!!**